In [1]:
#!/usr/bin/env python

from __future__ import print_function

import argparse
import h5py

from keras.models import model_from_json
import keras.backend as K

import os
import sys

sys.path.insert(0, os.path.abspath('..'))
os.chdir('../CoMET')

from evolutron.engine import DeepTrainer
from evolutron.tools import load_dataset, Handle, shape
from evolutron.motifs import motif_extraction

Using Theano backend.
ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 1 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [7]:
filename = 'models/zinc/100_10_12_50_krDeepCoDER.model'
hf = h5py.File(filename)
model_config = hf.attrs['model_config'].decode('utf8')
hf.close()

In [8]:
import evolutron.networks.krs.extra_layers as extra
custom_layers = {
    'Convolution1D': extra.Convolution1D,
    'Deconvolution1D': extra.Deconvolution1D,
    'MaxPooling1D': extra.MaxPooling1D,
    'Unpooling1D': extra.Unpooling1D,
    'Dense': extra.Dense,
    'Dedense': extra.Dedense,
    'Reshape': extra.Reshape,
    'Flatten': extra.Flatten
}

net = DeepTrainer(model_from_json(model_config, custom_objects=custom_layers))
net.load_all_param_values(filename)

handle = Handle.from_filename(filename)
data_id = handle.dataset

x_data = load_dataset(data_id, padded=False)

Dataset size: 11906


In [12]:
conv_layers = net.get_conv_layers()
conv_scores = conv_layers[0].output
custom_fun = K.function([net.inp], conv_scores)

In [18]:
import numpy as np
x_data = [np.expand_dims(x,0) for x in x_data]

In [33]:
res = custom_fun([x_data[0]])

In [34]:
res.shape

(1, 532, 100)